In [ ]:
#Imports
import numpy as np
import torch
import matplotlib.pyplot as plt
import csv
import cProfile
import pstats

from parameter_functions import distance_mse, distance_null, distance_discrete, get_prisoners_dilemma, get_stag_hunt, get_uniform_game, get_normal_game, Net
from evidential_cooperation import Agent, Environment, get_average_scores, test_net_cooperativity
from agent_generation import generate_twins, diverging_twins

In [ ]:
# Get average scores over multiple runs with agents trained on evidential cooperation
distance_f, distance_f_name = distance_mse, 'MSE'
game_generator, game_generator_name = get_prisoners_dilemma, 'PD'
agents_generator, agents_generator_name = diverging_twins, 'DIVERGING'
num_rounds, num_games, num_trials = 10000, 1, 1 # runtime is proportional to num_rounds * num_games * num_trials
death_rate = 0.0
net_n, agents_per_net = 3, 2 # number agents = net_n * agents_per_net
random_inputs = 5
lr = 0.001

average_scores, nets = get_average_scores(agents_generator=lambda: agents_generator(distance_f=distance_f, net_n=net_n, agents_per_net=agents_per_net, random_inputs=random_inputs, lr=lr), game_generator=game_generator, death_rate=death_rate, num_trials=num_trials, num_rounds=num_rounds, num_games=num_games, random_inputs=random_inputs, verbose=True)

# Plot and save the average scores
plt.clf()
plt.plot(average_scores)
plt.savefig('graphs/{}nets {}agents {} {} {} {}rounds {}games {}death {}lr {}random_inputs {}trials.png'.format(net_n, net_n*agents_per_net, game_generator_name, distance_f_name, agents_generator_name, num_rounds, num_games, death_rate, lr, random_inputs, num_trials))
plt.show()
with open('graphs/{}nets {}agents {} {} {} {}rounds {}games {}death {}lr {}random_inputs {}trials.csv'.format(net_n, net_n*agents_per_net, game_generator_name, distance_f_name, agents_generator_name, num_rounds, num_games, death_rate, lr, random_inputs, num_trials), 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(average_scores)

# Save the trained networks
for i, net in enumerate(nets):
    torch.save(net.state_dict(), 'models/net{} {}nets {}agents {} {} {} {}rounds {}games {}death {}lr {}random_inputs'.format(i, net_n, net_n*agents_per_net, game_generator_name, distance_f_name, agents_generator_name, num_rounds, num_games, death_rate, lr, random_inputs, num_trials))

print('Average score: {}'.format(sum(average_scores)/num_rounds))

In [ ]:
#Load nets
cooperativities = test_net_cooperativity(nets=nets, game_generator=get_prisoners_dilemma, random_inputs=random_inputs, randomize=False)
for i, cooperativity in enumerate(cooperativities):
    # Convert to numpy array
    cooperativity = np.array([c.detach().numpy() for c in cooperativity])
    #print('Cooperativity: {}'.format(cooperativity), type(cooperativity))
    # Plot the cooperativity and defectivity
    plt.clf()
    plt.plot(np.linspace(0, 2, 100), cooperativity[:, 0], label='Cooperativity')
    plt.plot(np.linspace(0, 2, 100), cooperativity[:, 1], label='Defectivity')
    plt.legend()
    plt.show()
    # Save the plots
    plt.savefig('graphs/{}nets {}agents {} {} {} {}rounds {}games {}death {}lr {}random_inputs cooperativity {}.png'.format(net_n, net_n*agents_per_net, game_generator_name, distance_f_name, agents_generator_name, num_rounds, num_games, death_rate, lr, random_inputs, i))


In [ ]:
type(average_scores)